In [27]:
# ============================================================
# ShadowSignals — Environmental Behavioural Intelligence (EBINT)
# Real-Time Covert-Intent Environmental Micro-Signal Engine
# Auto-refresh every 5 minutes (Option C)
# Dashboard + Log Feed (Option 3)
# Zero personal data, zero infrastructure cost, 100% open-source OSINT
# ============================================================

print("ShadowSignals Notebook Initialized — Part 1 Loaded.")


ShadowSignals Notebook Initialized — Part 1 Loaded.


In [26]:
!pip install --force-reinstall numpy==1.26.4 pandas==2.2.2 matplotlib==3.7.2 \
            scikit-learn==1.3.2 pytrends yt-dlp pytube \
            folium osmnx shapely geopy gtfs-realtime-bindings \
            aiohttp nest_asyncio tqdm


^C
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 4, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/main.py", line 11, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/u

In [28]:
import numpy as np
np.__version__


'1.26.4'

In [29]:
import torch
print("GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


GPU Available: True
GPU Name: Tesla P100-PCIE-16GB


In [30]:
import nest_asyncio
nest_asyncio.apply()
print("AsyncIO Patched for Kaggle")


AsyncIO Patched for Kaggle


In [31]:
import os

dirs = [
    "webcam_frames",
    "youtube_frames",
    "osmbase",
    "weather_cache",
    "transport_cache",
    "trends_cache",
    "shadow_logs"
]

for d in dirs:
    os.makedirs(d, exist_ok=True)

print("Directory structure created.")


Directory structure created.


In [32]:
import requests

def get_region_approx():
    try:
        r = requests.get("https://ipinfo.io/json", timeout=10).json()
        loc = r["loc"].split(",")
        lat, lon = float(loc[0]), float(loc[1])
        print("Auto-detected coordinates:", lat, lon)
        return lat, lon
    except:
        print("Auto-detection failed — please set manually below.")
        return None, None

LAT, LON = get_region_approx()


Auto-detected coordinates: 25.0531 121.5264


In [33]:
# Overwrite LAT/LON manually if you want EXACT region.
# Example:
# LAT = 23.0225
# LON = 72.5714

print("Active Coordinates:", LAT, LON)


Active Coordinates: 25.0531 121.5264


In [34]:
CONFIG = {
    "refresh_interval_sec": 300,        # Option C (5 minutes)
    "use_youtube": True,
    "use_webcams": True,
    "use_pytrends": True,
    "use_weather": True,
    "use_gtfs": True,
    "heavy_autoencoder": True,
    "log_limit": 100,
}

print("Runtime configuration loaded:")
for k,v in CONFIG.items():
    print(" -", k, "=", v)


Runtime configuration loaded:
 - refresh_interval_sec = 300
 - use_youtube = True
 - use_webcams = True
 - use_pytrends = True
 - use_weather = True
 - use_gtfs = True
 - heavy_autoencoder = True
 - log_limit = 100


In [35]:
# ===============================================
# PART 2 — IMPORTS & CORE UTILITIES
# ===============================================

import os
import cv2
import time
import json
import torch
import queue
import asyncio
import base64
import random
import logging
import requests
import numpy as np
import pandas as pd
from datetime import datetime
from io import BytesIO
from PIL import Image
from tqdm import tqdm
from pytrends.request import TrendReq

import aiohttp
from aiohttp import ClientSession

import matplotlib.pyplot as plt

# Optical Flow
from cv2 import calcOpticalFlowFarneback as farneback

print("Part 2 Imports Loaded.")


Part 2 Imports Loaded.


In [36]:
# ===============================================
# ASYNC SAFE FETCHER
# ===============================================

async def safe_get(session: ClientSession, url: str, timeout=10):
    try:
        async with session.get(url, timeout=timeout) as resp:
            if resp.status == 200:
                return await resp.read()
            else:
                return None
    except:
        return None

print("Async fetcher ready.")


Async fetcher ready.


In [37]:
# ===============================================
# IMAGE UTILITY
# ===============================================

def to_cv2_image(data: bytes):
    try:
        img = Image.open(BytesIO(data)).convert("RGB")
        return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    except:
        return None

print("Image utilities ready.")


Image utilities ready.


In [38]:
# ===============================================
# OPTICAL FLOW (Dense Farneback)
# ===============================================

def compute_optical_flow(prev_frame, next_frame):
    try:
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)
        flow = farneback(prev_gray, next_gray, None,
                         0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
        return np.mean(mag)
    except:
        return 0.0

print("Optical flow module ready.")


Optical flow module ready.


In [39]:
# ===============================================
# LOG STATE (Dashboard + Feed)
# ===============================================

SHADOW_LOG_FEED = []
MAX_LOGS = CONFIG["log_limit"]

def add_log(entry: dict):
    SHADOW_LOG_FEED.append(entry)
    if len(SHADOW_LOG_FEED) > MAX_LOGS:
        SHADOW_LOG_FEED.pop(0)

print("Logging system initialized.")


Logging system initialized.


In [40]:
# ===============================================
# DASHBOARD RENDERER
# ===============================================

from IPython.display import clear_output, display

def render_dashboard(last_entry):
    clear_output(wait=True)
    
    print("=====================================================")
    print("                SHADOWSIGNALS DASHBOARD               ")
    print("=====================================================")
    print(f"Timestamp: {last_entry['timestamp']}")
    print(f"ShadowScore: {last_entry['shadow_score']:.2f} / 100")
    print("-----------------------------------------------------")
    print("Top Signals:")
    for key, val in last_entry["signals"].items():
        print(f" - {key}: {val:.4f}")
    print("-----------------------------------------------------")
    print("Anomaly Score:", last_entry["anomaly_score"])
    print("Regional Baseline Shift:", last_entry["baseline_shift"])
    print("=====================================================")
    
    # Log Feed (Last 5 Cycles)
    print("\nRecent Cycles (Last 5):")
    for e in SHADOW_LOG_FEED[-5:]:
        print(f"[{e['timestamp']}] Score={e['shadow_score']:.2f}")

print("Dashboard system ready.")


Dashboard system ready.


In [41]:
# ===============================================
# PYTRENDS GLOBAL INSTANCE
# ===============================================

pytrends = TrendReq(hl='en-US', tz=330)
print("PyTrends instance ready.")


PyTrends instance ready.


In [42]:
# ============================================================
# PUBLIC WEBCAM FRAME FETCHER
# ============================================================

async def fetch_webcam_frame(url: str):
    """
    Fetches a single frame from a direct-image public webcam URL.
    Must be a JPG/PNG image endpoint, not HTML.
    """
    async with aiohttp.ClientSession() as session:
        data = await safe_get(session, url)
        if data is None:
            return None
        frame = to_cv2_image(data)
        return frame

print("Webcam fetcher ready.")


Webcam fetcher ready.


In [43]:
# ============================================================
# YOUTUBE LIVE STREAM – SINGLE FRAME EXTRACTOR
# ============================================================

import subprocess
import uuid

def fetch_youtube_frame(url: str):
    """
    Extracts a single frame from a YouTube live stream using yt-dlp.
    Works in Kaggle.
    """
    try:
        temp_file = f"youtube_frames/{uuid.uuid4()}.jpg"
        cmd = [
            "yt-dlp",
            "-o", temp_file,
            "--skip-download",
            "--write-thumbnail",
            url
        ]
        subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        if not os.path.exists(temp_file):
            return None

        img = cv2.imread(temp_file)
        return img
    except Exception as e:
        return None

print("YouTube frame extractor ready.")


YouTube frame extractor ready.


In [44]:
# ============================================================
# WEATHER FETCHER — OPEN METEO
# ============================================================

def fetch_weather(lat, lon):
    try:
        url = (
            f"https://api.open-meteo.com/v1/forecast?"
            f"latitude={lat}&longitude={lon}&current_weather=true"
        )
        r = requests.get(url, timeout=10).json()
        cw = r.get("current_weather", {})
        return {
            "temperature": cw.get("temperature", 0),
            "windspeed": cw.get("windspeed", 0),
            "winddir": cw.get("winddirection", 0),
            "weathercode": cw.get("weathercode", 0),
        }
    except:
        return {
            "temperature": 0,
            "windspeed": 0,
            "winddir": 0,
            "weathercode": 0,
        }

print("Weather module ready.")


Weather module ready.


In [45]:
# ============================================================
# PYTRENDS REAL-TIME TREND SPIKE FETCHER
# ============================================================

def fetch_trends(keyword="security"):
    try:
        pytrends.build_payload([keyword], cat=0, timeframe="now 1-H", geo="")
        df = pytrends.interest_over_time()
        if df.empty:
            return 0.0
        return float(df[keyword].iloc[-1])
    except:
        return 0.0

print("PyTrends fetcher ready.")


PyTrends fetcher ready.


In [46]:
# ============================================================
# GTFS REALTIME DELAY FETCHER
# ============================================================

from google.transit import gtfs_realtime_pb2

def fetch_gtfs_delay(url: str):
    """
    Returns average delay (in seconds) from a GTFS feed.
    If feed unavailable, returns 0.
    """
    try:
        feed = gtfs_realtime_pb2.FeedMessage()
        resp = requests.get(url, timeout=10)
        feed.ParseFromString(resp.content)

        delays = []
        for entity in feed.entity:
            if entity.HasField("trip_update"):
                for upd in entity.trip_update.stop_time_update:
                    if upd.HasField("arrival") and upd.arrival.HasField("delay"):
                        delays.append(upd.arrival.delay)

        if len(delays) == 0:
            return 0.0

        return float(np.mean(delays))
    except:
        return 0.0

print("GTFS delay module ready.")


GTFS delay module ready.


In [47]:
# ============================================================
# NIGHT-LIGHT INTENSITY EXTRACTOR
# ============================================================

def compute_night_light(frame):
    """
    Computes normalized brightness: 0 (dark) → 1 (bright).
    """
    try:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        return float(np.mean(gray) / 255.0)
    except:
        return 0.0

print("Night-light extractor ready.")


Night-light extractor ready.


In [48]:
# ============================================================
# TRAFFIC DENSITY ESTIMATOR
# ============================================================

def estimate_traffic_density(frame):
    try:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 50, 150)
        density = np.sum(edges) / (edges.size * 255)
        return float(density)
    except:
        return 0.0

print("Traffic density module ready.")


Traffic density module ready.


In [49]:
# ============================================================
# CROWD FLOW FROM SEQUENTIAL FRAMES
# ============================================================

LAST_FRAME = None

def compute_crowd_flow(frame):
    global LAST_FRAME
    if LAST_FRAME is None:
        LAST_FRAME = frame
        return 0.0

    flow_val = compute_optical_flow(LAST_FRAME, frame)
    LAST_FRAME = frame
    return float(flow_val)

print("Crowd-flow module ready.")


Crowd-flow module ready.


In [50]:
# ============================================================
# NORMALIZATION & UTILITY SIGNAL FUNCTIONS
# ============================================================

def normalize(value, min_v=0, max_v=1):
    """
    Normalize a scalar to 0–1 range.
    """
    try:
        if value < min_v: return 0.0
        if value > max_v: return 1.0
        return (value - min_v) / (max_v - min_v)
    except:
        return 0.0

def safe_mean(arr):
    arr = [x for x in arr if x is not None]
    if len(arr) == 0:
        return 0.0
    return float(np.mean(arr))
    
print("Normalization utilities ready.")


Normalization utilities ready.


In [51]:
# ============================================================
# ENTROPY MODELLING (Shannon Entropy)
# ============================================================

def shannon_entropy(arr):
    """
    Compute Shannon entropy of signal distribution.
    """
    try:
        arr = np.array(arr)
        if len(arr) == 0:
            return 0.0
        histogram, _ = np.histogram(arr, bins=10, density=True)
        histogram = histogram[histogram > 0]
        return float(-np.sum(histogram * np.log2(histogram)))
    except:
        return 0.0

print("Entropy modelling ready.")


Entropy modelling ready.


In [52]:
# ============================================================
# DIVERSITY WEIGHTING
# ============================================================

def diversity_weight(signal_name):
    """
    Assign weights so that no modality dominates.
    """
    weights = {
        "traffic_density": 1.0,
        "crowd_flow": 1.2,
        "night_light": 0.8,
        "weather_temperature": 0.5,
        "weather_windspeed": 0.5,
        "weather_code": 0.4,
        "trend_score": 1.3,
        "transport_delay": 1.1,
    }
    return weights.get(signal_name, 1.0)

print("Diversity weighting ready.")


Diversity weighting ready.


In [53]:
# ============================================================
# OSM REGION BASELINE MODELLING
# ============================================================

import osmnx as ox

OSM_BASELINE = None

def build_osm_baseline(lat, lon):
    """
    Build regional baseline from OSM:
    - Street density
    - Intersection count
    - Building footprint density
    """
    global OSM_BASELINE
    
    print("Building OSM baseline... (may take 30–60 seconds)")
    try:
        G = ox.graph_from_point((lat, lon), dist=1500, network_type='drive')
        nodes, edges = ox.graph_to_gdfs(G)
        
        street_density = edges.length.sum() / (math.pi * (1500**2))
        intersection_count = len(nodes)
        
        # Building footprints
        buildings = ox.geometries_from_point((lat, lon), tags={"building": True}, dist=1500)
        building_density = len(buildings)
        
        OSM_BASELINE = {
            "street_density": float(street_density),
            "intersection_count": float(intersection_count),
            "building_density": float(building_density),
        }
        
        print("OSM baseline ready.")
        return OSM_BASELINE
    except Exception as e:
        print("OSM baseline error:", e)
        OSM_BASELINE = {
            "street_density": 0.5,
            "intersection_count": 50,
            "building_density": 50,
        }
        return OSM_BASELINE

print("OSM baseline module ready.")


OSM baseline module ready.


In [54]:
# ============================================================
# ROLLING BASELINE FOR TIME-SERIES NORMALIZATION
# ============================================================

BASELINE_HISTORY = {}

def update_rolling_baseline(name, value):
    if name not in BASELINE_HISTORY:
        BASELINE_HISTORY[name] = []
    BASELINE_HISTORY[name].append(value)
    if len(BASELINE_HISTORY[name]) > 144:  # ~12 hours @5min intervals
        BASELINE_HISTORY[name].pop(0)

def compute_baseline_shift(name, value):
    if name not in BASELINE_HISTORY or len(BASELINE_HISTORY[name]) < 5:
        return 0.0
    baseline_mean = np.mean(BASELINE_HISTORY[name])
    return float(value - baseline_mean)

print("Rolling baseline engine ready.")


Rolling baseline engine ready.


In [55]:
# ============================================================
# MASTER SIGNAL EXTRACTOR (20+ EBINT SIGNALS)
# ============================================================

def extract_signals(frame, weather, trend_val, gtfs_delay):
    """
    Convert environmental observations into 20+ fused signals.
    """
    signals = {}

    # Webcam-based signals
    traffic = estimate_traffic_density(frame)
    crowd = compute_crowd_flow(frame)
    night = compute_night_light(frame)

    # Weather signals
    temp = weather["temperature"]
    wind = weather["windspeed"]
    code = weather["weathercode"]

    # Entropy signals (use history)
    update_rolling_baseline("traffic", traffic)
    update_rolling_baseline("crowd_flow", crowd)
    update_rolling_baseline("night_light", night)
    update_rolling_baseline("trend", trend_val)
    update_rolling_baseline("transport_delay", gtfs_delay)

    signals = {
        "traffic_density": traffic,
        "crowd_flow": crowd,
        "night_light": night,
        "weather_temperature": temp,
        "weather_windspeed": wind,
        "weather_code": code,
        "trend_score": trend_val,
        "transport_delay": gtfs_delay,

        # Entropy based signals
        "entropy_traffic": shannon_entropy(BASELINE_HISTORY.get("traffic", [])),
        "entropy_crowd": shannon_entropy(BASELINE_HISTORY.get("crowd_flow", [])),
        "entropy_night": shannon_entropy(BASELINE_HISTORY.get("night_light", [])),
        "entropy_trend": shannon_entropy(BASELINE_HISTORY.get("trend", [])),
        "entropy_transport": shannon_entropy(BASELINE_HISTORY.get("transport_delay", [])),

        # Baseline shifts
        "shift_traffic": compute_baseline_shift("traffic", traffic),
        "shift_crowd": compute_baseline_shift("crowd_flow", crowd),
        "shift_night": compute_baseline_shift("night_light", night),

        # Diversity weighted signals
        "weighted_traffic": traffic * diversity_weight("traffic_density"),
        "weighted_crowd": crowd * diversity_weight("crowd_flow"),
        "weighted_trend": trend_val * diversity_weight("trend_score"),
    }

    return signals

print("Master EBINT signal extractor ready.")


Master EBINT signal extractor ready.


In [56]:
# ============================================================
# SHADOWSIGNAL VECTOR FEATURE ORDER
# ============================================================

SHADOW_FEATURES = [
    "traffic_density",
    "crowd_flow",
    "night_light",

    "weather_temperature",
    "weather_windspeed",
    "weather_code",

    "trend_score",
    "transport_delay",

    # Entropy signals
    "entropy_traffic",
    "entropy_crowd",
    "entropy_night",
    "entropy_trend",
    "entropy_transport",

    # Baseline shifts
    "shift_traffic",
    "shift_crowd",
    "shift_night",

    # Diversity-weighted modalities
    "weighted_traffic",
    "weighted_crowd",
    "weighted_trend",
]

print("ShadowSignal feature list loaded. Total features:", len(SHADOW_FEATURES))


ShadowSignal feature list loaded. Total features: 19


In [57]:
# ============================================================
# NORMALIZATION RANGES FOR EACH FEATURE
# ============================================================

NORMALIZATION_RULES = {
    "traffic_density": (0, 0.1),
    "crowd_flow": (0, 2.0),
    "night_light": (0, 1.0),

    "weather_temperature": (-10, 45),
    "weather_windspeed": (0, 80),
    "weather_code": (0, 100),

    "trend_score": (0, 100),
    "transport_delay": (-300, 300),

    "entropy_traffic": (0, 3),
    "entropy_crowd": (0, 3),
    "entropy_night": (0, 3),
    "entropy_trend": (0, 3),
    "entropy_transport": (0, 3),

    "shift_traffic": (-0.05, 0.05),
    "shift_crowd": (-0.5, 0.5),
    "shift_night": (-0.3, 0.3),

    "weighted_traffic": (0, 0.2),
    "weighted_crowd": (0, 3.0),
    "weighted_trend": (0, 200),
}

print("Normalization rules loaded.")


Normalization rules loaded.


In [58]:
# ============================================================
# SHADOWSIGNAL VECTOR BUILDER
# ============================================================

def build_shadow_vector(signal_dict):
    """
    Convert a signal dictionary into a normalized ML-ready vector.
    Returns: np.array of shape (N,)
    """
    vector = []

    for key in SHADOW_FEATURES:
        value = signal_dict.get(key, 0.0)
        min_v, max_v = NORMALIZATION_RULES.get(key, (0, 1))
        norm_val = normalize(value, min_v, max_v)
        vector.append(norm_val)

    return np.array(vector, dtype=np.float32)

print("ShadowSignal Vector builder ready.")


ShadowSignal Vector builder ready.


In [59]:
# ============================================================
# OPTIONAL: VECTOR DEBUG PRINTER
# ============================================================

def print_shadow_vector(vec):
    print("ShadowSignal Vector:")
    for i, key in enumerate(SHADOW_FEATURES):
        print(f"{i:02d}. {key:20s} = {vec[i]:.4f}")

print("Vector debug printer ready.")


Vector debug printer ready.


In [60]:
# ============================================================
# TEST VECTOR BUILDING (Example)
# ============================================================

# Fake example data (replace with real pipeline later)
test_signals = {
    "traffic_density": 0.03,
    "crowd_flow": 0.7,
    "night_light": 0.4,
    "weather_temperature": 30,
    "weather_windspeed": 10,
    "weather_code": 2,
    "trend_score": 65,
    "transport_delay": 25,
    "entropy_traffic": 1.0,
    "entropy_crowd": 0.6,
    "entropy_night": 1.3,
    "entropy_trend": 0.8,
    "entropy_transport": 1.2,
    "shift_traffic": 0.002,
    "shift_crowd": -0.03,
    "shift_night": 0.1,
    "weighted_traffic": 0.05,
    "weighted_crowd": 0.8,
    "weighted_trend": 60,
}

test_vec = build_shadow_vector(test_signals)
print_shadow_vector(test_vec)
print("Vector length:", len(test_vec))


ShadowSignal Vector:
00. traffic_density      = 0.3000
01. crowd_flow           = 0.3500
02. night_light          = 0.4000
03. weather_temperature  = 0.7273
04. weather_windspeed    = 0.1250
05. weather_code         = 0.0200
06. trend_score          = 0.6500
07. transport_delay      = 0.5417
08. entropy_traffic      = 0.3333
09. entropy_crowd        = 0.2000
10. entropy_night        = 0.4333
11. entropy_trend        = 0.2667
12. entropy_transport    = 0.4000
13. shift_traffic        = 0.5200
14. shift_crowd          = 0.4700
15. shift_night          = 0.6667
16. weighted_traffic     = 0.2500
17. weighted_crowd       = 0.2667
18. weighted_trend       = 0.3000
Vector length: 19


In [61]:
# ============================================================
# TORCH DEVICE SETUP
# ============================================================

import torch
import torch.nn as nn
import torch.optim as optim

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)


Using device: cuda


In [62]:
# ============================================================
# HEAVY DEEP AUTOENCODER MODEL
# ============================================================

class ShadowAutoencoder(nn.Module):
    def __init__(self, input_dim):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.SiLU(),
            nn.Dropout(0.1),
            
            nn.Linear(64, 48),
            nn.SiLU(),
            nn.Dropout(0.1),

            nn.Linear(48, 32),
            nn.SiLU(),

            nn.Linear(32, 16),
            nn.SiLU(),

            nn.Linear(16, 8),   # bottleneck
            nn.SiLU(),
        )

        self.decoder = nn.Sequential(
            nn.Linear(8, 16),
            nn.SiLU(),
            
            nn.Linear(16, 32),
            nn.SiLU(),

            nn.Linear(32, 48),
            nn.SiLU(),

            nn.Linear(48, 64),
            nn.SiLU(),

            nn.Linear(64, input_dim),
            nn.Sigmoid(),  # because inputs are normalized to 0–1
        )

    def forward(self, x):
        z = self.encoder(x)
        out = self.decoder(z)
        return out


print("Heavy ShadowAutoencoder class ready.")


Heavy ShadowAutoencoder class ready.


In [63]:
# ============================================================
# INITIALIZE AUTOENCODER
# ============================================================

INPUT_DIM = len(SHADOW_FEATURES)   # from Part 5

autoencoder = ShadowAutoencoder(INPUT_DIM).to(DEVICE)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

print("Autoencoder initialized with input dim:", INPUT_DIM)


Autoencoder initialized with input dim: 19


In [64]:
# ============================================================
# TRAINING BUFFER FOR ONLINE LEARNING
# ============================================================

TRAIN_BUFFER = []   # stores normalized vectors
MAX_BUFFER = 2000   # limit to avoid excessive memory use

def add_to_buffer(vec):
    global TRAIN_BUFFER
    TRAIN_BUFFER.append(vec)
    if len(TRAIN_BUFFER) > MAX_BUFFER:
        TRAIN_BUFFER.pop(0)

print("Training buffer ready.")


Training buffer ready.


In [72]:
# ============================================================
# HEAVY GPU TRAINING FUNCTION
# ============================================================

def train_autoencoder(epochs=5, batch_size=16):
    if len(TRAIN_BUFFER) < 200:
        print("Not enough data to train (need 200+ vectors).")
        return []
    
    print(f"Training autoencoder on {len(TRAIN_BUFFER)} samples...")

    # FIX: Convert list → single NumPy array → tensor
    np_data = np.array(TRAIN_BUFFER, dtype=np.float32)
    data = torch.tensor(np_data, dtype=torch.float32).to(DEVICE)

    dataset = torch.utils.data.TensorDataset(data)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    losses = []
    autoencoder.train()

    for epoch in range(epochs):
        epoch_loss = 0.0

        for batch in loader:
            optimizer.zero_grad()
            x = batch[0]

            z = autoencoder.encoder(x)
            x_recon = autoencoder.decoder(z)

            loss = criterion(x_recon, x)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        losses.append(epoch_loss)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f}")

    print("Training complete.")
    return losses

In [73]:
# ============================================================
# ANOMALY SCORE CALCULATION
# ============================================================

def compute_anomaly_score(vec):
    autoencoder.eval()
    with torch.no_grad():
        x = torch.tensor(vec, dtype=torch.float32).to(DEVICE)
        recon = autoencoder(x)
        loss = torch.mean((recon - x)**2).item()
    return float(loss)


In [74]:
# ============================================================
# OPTIONAL TEST — SANITY CHECK
# ============================================================

# Create synthetic normal vectors
for _ in range(500):
    fake_vec = np.clip(np.random.normal(0.5, 0.1, INPUT_DIM), 0, 1).astype(np.float32)
    add_to_buffer(fake_vec)

print("Added 500 synthetic vectors to buffer.")

# Train once to test
losses = train_autoencoder(epochs=5)

print("Loss curve:", losses)


Added 500 synthetic vectors to buffer.
Training autoencoder on 1000 samples...
Epoch 1/5 - Loss: 0.6385
Epoch 2/5 - Loss: 0.6385
Epoch 3/5 - Loss: 0.6388
Epoch 4/5 - Loss: 0.6381
Epoch 5/5 - Loss: 0.6255
Training complete.
Loss curve: [0.6385182840749621, 0.6384512009099126, 0.6387604093179107, 0.6380912233144045, 0.6255325991660357]


In [76]:
# ============================================================
# SHADOWGRAPH FUSION ENGINE
# ============================================================

def shadowgraph_fusion(signals, anomaly_score):
    """
    Core EBINT fusion engine.
    Combines:
    - Raw signals
    - Entropy signals
    - Baseline shifts
    - Diversity weights
    - Autoencoder anomaly score
    """

    # Weight anomaly score (strongest indicator)
    w_anomaly = anomaly_score * 20

    # Weighted multi-signal fusion
    w_traffic  = signals["weighted_traffic"]
    w_crowd    = signals["weighted_crowd"]
    w_trend    = signals["weighted_trend"]

    # Entropy shifts
    e_traffic  = signals["entropy_traffic"]
    e_crowd    = signals["entropy_crowd"]
    e_night    = signals["entropy_night"]
    e_trend    = signals["entropy_trend"]

    # Baseline deviations
    s_traffic = abs(signals["shift_traffic"])
    s_crowd   = abs(signals["shift_crowd"])
    s_night   = abs(signals["shift_night"])

    # Composite score (raw 0–something)
    raw_score = (
        w_anomaly
        + 10 * w_traffic
        + 10 * w_crowd
        + 5  * w_trend
        + 3  * e_traffic
        + 3  * e_crowd
        + 2  * e_night
        + 2  * e_trend
        + 15 * s_traffic
        + 15 * s_crowd
        + 10 * s_night
    )

    return float(raw_score)


In [77]:
# ============================================================
# SHADOWSCORE NORMALIZATION (0–100)
# ============================================================

def compute_shadow_score(raw):
    # Sigmoid-like squashing + scaling to 0-100
    stabilized = np.tanh(raw / 25)   # compress
    score = np.clip((stabilized + 1) * 50, 0, 100)
    return float(score)


In [78]:
# ============================================================
# SHADOWSIGNALS — FULL PIPELINE FOR ONE CYCLE
# ============================================================

async def run_shadow_cycle(webcam_url=None, youtube_url=None, gtfs_url=None):
    # --------------------------------------------
    # 1. FRAME ACQUISITION
    # --------------------------------------------
    frame = None

    if webcam_url:
        frame = await fetch_webcam_frame(webcam_url)

    # fallback to YouTube frame if no webcam
    if frame is None and youtube_url:
        frame = fetch_youtube_frame(youtube_url)

    # If still no frame: create dummy
    if frame is None:
        frame = np.zeros((480, 640, 3), dtype=np.uint8)

    # --------------------------------------------
    # 2. WEATHER
    # --------------------------------------------
    weather = fetch_weather(LAT, LON)

    # --------------------------------------------
    # 3. PYTRENDS
    # --------------------------------------------
    trend_val = fetch_trends("security")

    # --------------------------------------------
    # 4. GTFS DELAY
    # --------------------------------------------
    gtfs_delay = 0.0
    if gtfs_url:
        gtfs_delay = fetch_gtfs_delay(gtfs_url)

    # --------------------------------------------
    # 5. EXTRACT EBINT SIGNALS
    # --------------------------------------------
    signals = extract_signals(frame, weather, trend_val, gtfs_delay)

    # --------------------------------------------
    # 6. ShadowSignal Vector
    # --------------------------------------------
    vec = build_shadow_vector(signals)
    add_to_buffer(vec)  # add to training buffer

    # --------------------------------------------
    # 7. Autoencoder Anomaly Score
    # --------------------------------------------
    anomaly = compute_anomaly_score(vec)

    # --------------------------------------------
    # 8. ShadowGraph Fusion
    # --------------------------------------------
    raw = shadowgraph_fusion(signals, anomaly)
    score = compute_shadow_score(raw)

    # --------------------------------------------
    # 9. Log Entry
    # --------------------------------------------
    entry = {
        "timestamp": datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC"),
        "shadow_score": score,
        "anomaly_score": anomaly,
        "signals": signals,
        "baseline_shift": {
            "traffic": signals["shift_traffic"],
            "crowd": signals["shift_crowd"],
            "night": signals["shift_night"],
        }
    }

    add_log(entry)  # update feed
    render_dashboard(entry)  # show dashboard

    return entry


In [79]:
# ============================================================
# OPTIONAL MANUAL TEST OF ONE SHADOW CYCLE
# ============================================================

# Example webcam (replace with your real one)
test_webcam = None

# Example YouTube live stream (replace with your real cam)
test_youtube = None

# Example GTFS feed
test_gtfs = None

entry = await run_shadow_cycle(
    webcam_url=test_webcam,
    youtube_url=test_youtube,
    gtfs_url=test_gtfs
)

print("ShadowScore =", entry["shadow_score"])


                SHADOWSIGNALS DASHBOARD               
Timestamp: 2025-11-27 16:08:04 UTC
ShadowScore: 100.00 / 100
-----------------------------------------------------
Top Signals:
 - traffic_density: 0.0000
 - crowd_flow: 0.0000
 - night_light: 0.0000
 - weather_temperature: 17.5000
 - weather_windspeed: 16.6000
 - weather_code: 3.0000
 - trend_score: 100.0000
 - transport_delay: 0.0000
 - entropy_traffic: -33.2193
 - entropy_crowd: -33.2193
 - entropy_night: -33.2193
 - entropy_trend: -33.2193
 - entropy_transport: -33.2193
 - shift_traffic: 0.0000
 - shift_crowd: 0.0000
 - shift_night: 0.0000
 - weighted_traffic: 0.0000
 - weighted_crowd: 0.0000
 - weighted_trend: 130.0000
-----------------------------------------------------
Anomaly Score: 0.1557055413722992
Regional Baseline Shift: {'traffic': 0.0, 'crowd': 0.0, 'night': 0.0}

Recent Cycles (Last 5):
[2025-11-27 16:08:04 UTC] Score=100.00
ShadowScore = 99.99999999929197


In [80]:
# ============================================================
# ASYNC SLEEP HELPER (Kaggle-safe)
# ============================================================

async def async_sleep(seconds):
    for _ in range(seconds):
        await asyncio.sleep(1)


In [81]:
# ============================================================
# SHADOWSIGNALS — REAL-TIME AUTO LOOP (5 MIN)
# ============================================================

async def shadowsignals_auto_loop(
    webcam_url=None,
    youtube_url=None,
    gtfs_url=None,
    interval=CONFIG["refresh_interval_sec"]
):
    print("=====================================================")
    print("  SHADOWSIGNALS AUTO LOOP STARTED (REAL-TIME MODE)   ")
    print("  Refresh Interval:", interval, "seconds")
    print("  Press STOP/INTERRUPT to end.")
    print("=====================================================\n")

    cycle = 0

    while True:
        cycle += 1
        print(f"\n[Cycle {cycle}] Running ShadowSignals Engine...\n")

        # Run one complete EBINT cycle
        try:
            await run_shadow_cycle(
                webcam_url=webcam_url,
                youtube_url=youtube_url,
                gtfs_url=gtfs_url
            )
        except Exception as e:
            print("Cycle error:", e)

        print(f"[Cycle {cycle}] Completed. Waiting {interval} seconds...\n")

        # Wait for next cycle
        await async_sleep(interval)


In [82]:
import re
import requests

def extract_insecam_snapshot(insecam_url):
    print("Fetching page...")
    r = requests.get(insecam_url, timeout=10)
    html = r.text

    # Extract IP address from internal script
    ip_matches = re.findall(r'http[s]?://([\d\.]+):(\d+)', html)
    if not ip_matches:
        print("No IP found in the page. Camera likely offline.")
        return None

    ip, port = ip_matches[0]
    base = f"http://{ip}:{port}"

    print("Candidate camera base:", base)

    # Try most common snapshot endpoints
    paths = [
        "/snapshot.jpg",
        "/shot.jpg",
        "/image.jpg",
        "/cgi-bin/snapshot.cgi",
        "/axis-cgi/jpg/image.cgi",
        "/jpg/image.jpg",
        "/video.jpg"
    ]

    for p in paths:
        test_url = base + p
        try:
            r = requests.get(test_url, timeout=5)
            if r.status_code == 200 and r.headers.get("content-type", "").startswith("image"):
                print("LIVE SNAPSHOT FOUND:", test_url)
                return test_url
        except:
            pass

    print("No working snapshot endpoint found — camera offline or protected.")
    return None


In [83]:
url = extract_insecam_snapshot("http://www.insecam.org/en/view/261001/")
print("Result:", url)


Fetching page...
No IP found in the page. Camera likely offline.
Result: None


In [85]:
WINDY_CAMERAS = [
    "https://images-webcams.windy.com/90/1516793090/current.jpg",  # Taj Mahal
    "https://images-webcams.windy.com/89/1516788189/current.jpg",  # Marine Drive Mumbai
    "https://images-webcams.windy.com/62/1516789362/current.jpg",  # Goa Beach
    "https://images-webcams.windy.com/47/1516790347/current.jpg",  # Manali Himalayas
    "https://images-webcams.windy.com/10/1516789610/current.jpg",  # Leh Market
]


In [86]:
import re
import requests

def extract_insecam_snapshot(insecam_url):
    try:
        r = requests.get(insecam_url, timeout=10)
        html = r.text
    except:
        return None
    
    # Extract IP + port from page
    matches = re.findall(r'src="(http[^"]+)"', html)
    candidates = [m for m in matches if "jpg" in m or "cgi" in m]

    # Try common snapshot paths if IP found
    ip_matches = re.findall(r'http[s]?://([\d\.]+):(\d+)', html)
    if ip_matches:
        ip, port = ip_matches[0]
        base = f"http://{ip}:{port}"
        paths = [
            "/snapshot.jpg",
            "/shot.jpg",
            "/image.jpg",
            "/cgi-bin/snapshot.cgi",
            "/axis-cgi/jpg/image.cgi",
            "/jpg/image.jpg"
        ]
        candidates += [base + p for p in paths]

    # Validate candidates
    for url in candidates:
        try:
            r = requests.get(url, timeout=5)
            if r.status_code == 200 and "image" in r.headers.get("content-type", ""):
                return url
        except:
            pass

    return None


In [87]:
INSECAM_PAGES = [
    "http://www.insecam.org/en/bycountry/IN/"  # Country page (auto-scan)
]


In [88]:
async def resolve_live_camera():
    # 1. Try Windy first
    for url in WINDY_CAMERAS:
        try:
            r = requests.get(url, timeout=5)
            if r.status_code == 200:
                return url
        except:
            pass

    # 2. If Windy failed → try Insecam
    for page in INSECAM_PAGES:
        extracted = extract_insecam_snapshot(page)
        if extracted:
            return extracted

    # 3. If everything fails → fallback to black frame
    return None


In [91]:
# ============================================================
# MULTI-CAMERA FALLBACK CONFIGURATION
# ============================================================

MULTI_CAMERA_LIST = [
    # Add as many as you want (direct JPG/PNG endpoints)

    "https://images-webcams.windy.com/90/1516793090/current.jpg",   # Taj Mahal
    "https://images-webcams.windy.com/89/1516788189/current.jpg",   # Marine Drive, Mumbai
    "https://images-webcams.windy.com/62/1516789362/current.jpg",   # Goa Beach
    "https://images-webcams.windy.com/47/1516790347/current.jpg",   # Manali
    "https://images-webcams.windy.com/10/1516789610/current.jpg",   # Leh/Ladakh

    # Add more here if you find new ones
]


In [92]:
# ============================================================
# MULTI-CAMERA LIVE SNAPSHOT RESOLVER
# ============================================================

async def resolve_live_camera():
    """
    Tries each camera in MULTI_CAMERA_LIST until a working frame is found.
    Returns: working camera URL or None
    """
    for cam_url in MULTI_CAMERA_LIST:
        try:
            frame = await fetch_webcam_frame(cam_url)
            if frame is not None:
                print("Live camera found:", cam_url)
                return cam_url
        except:
            pass

    print("No live camera found. Using fallback black frame.")
    return None


In [93]:
# ============================================================
# START REAL-TIME AUTOLOOP WITH MULTI-CAMERA FALLBACK
# ============================================================

resolved_cam = await resolve_live_camera()
print("Using Camera:", resolved_cam)

await shadowsignals_auto_loop(
    webcam_url=resolved_cam,
    youtube_url=None,
    gtfs_url=None,
    interval=300
)


                SHADOWSIGNALS DASHBOARD               
Timestamp: 2025-11-27 17:08:24 UTC
ShadowScore: 100.00 / 100
-----------------------------------------------------
Top Signals:
 - traffic_density: 0.0000
 - crowd_flow: 0.0000
 - night_light: 0.0000
 - weather_temperature: 17.6000
 - weather_windspeed: 16.3000
 - weather_code: 3.0000
 - trend_score: 95.0000
 - transport_delay: 0.0000
 - entropy_traffic: -33.2193
 - entropy_crowd: -33.2193
 - entropy_night: -33.2193
 - entropy_trend: 1.4692
 - entropy_transport: -33.2193
 - shift_traffic: 0.0000
 - shift_crowd: 0.0000
 - shift_night: 0.0000
 - weighted_traffic: 0.0000
 - weighted_crowd: 0.0000
 - weighted_trend: 123.5000
-----------------------------------------------------
Anomaly Score: 0.1400323212146759
Regional Baseline Shift: {'traffic': 0.0, 'crowd': 0.0, 'night': 0.0}

Recent Cycles (Last 5):
[2025-11-27 16:08:04 UTC] Score=100.00
[2025-11-27 16:58:48 UTC] Score=100.00
[2025-11-27 17:03:50 UTC] Score=100.00
[2025-11-27 17:0

CancelledError: 

In [95]:
# ============================================================
# MULTI-CAMERA FALLBACK LIST
# Put as many as you want (JPG snapshot URLs only)
# ============================================================

WEBCAM_LIST = [
    # Add your cameras here ↓↓↓
    "https://images-webcams.windy.com/90/1516793090/current.jpg",   # Taj Mahal
    "https://images-webcams.windy.com/89/1516788189/current.jpg",   # Marine Drive
    "https://images-webcams.windy.com/62/1516789362/current.jpg",   # Goa Beach
    "https://images-webcams.windy.com/47/1516790347/current.jpg",   # Manali
]


In [96]:
# ============================================================
# MULTI-CAMERA RESOLVER
# ============================================================

async def resolve_live_camera():
    """
    Try each camera in WEBCAM_LIST until one returns a valid frame.
    """
    print("\nResolving live camera source...")
    for cam_url in WEBCAM_LIST:
        print(f"Trying: {cam_url}")
        try:
            frame = await fetch_webcam_frame(cam_url)
            if frame is not None:
                print(f"✔ Camera online: {cam_url}")
                return cam_url
        except:
            pass

    print("✖ No cameras available. Using fallback black frame.")
    return None


In [ ]:
# ============================================================
# START SHADOWSIGNALS AUTO LOOP (with multi-camera fallback)
# ============================================================

resolved_cam = await resolve_live_camera()
print("Using Camera:", resolved_cam)

await shadowsignals_auto_loop(
    webcam_url=resolved_cam,
    youtube_url=None,   # YouTube fallback disabled OR add your list
    gtfs_url=None,      # India has no open GTFS, so keep None
    interval=300        # 5-minute EBINT cycle
)


                SHADOWSIGNALS DASHBOARD               
Timestamp: 2025-11-27 17:17:48 UTC
ShadowScore: 100.00 / 100
-----------------------------------------------------
Top Signals:
 - traffic_density: 0.0000
 - crowd_flow: 0.0000
 - night_light: 0.0000
 - weather_temperature: 17.6000
 - weather_windspeed: 16.8000
 - weather_code: 3.0000
 - trend_score: 89.0000
 - transport_delay: 0.0000
 - entropy_traffic: -33.2193
 - entropy_crowd: -33.2193
 - entropy_night: -33.2193
 - entropy_trend: 1.8178
 - entropy_transport: -33.2193
 - shift_traffic: 0.0000
 - shift_crowd: 0.0000
 - shift_night: 0.0000
 - weighted_traffic: 0.0000
 - weighted_crowd: 0.0000
 - weighted_trend: 115.7000
-----------------------------------------------------
Anomaly Score: 0.1376011222600937
Regional Baseline Shift: {'traffic': 0.0, 'crowd': 0.0, 'night': 0.0}

Recent Cycles (Last 5):
[2025-11-27 16:58:48 UTC] Score=100.00
[2025-11-27 17:03:50 UTC] Score=100.00
[2025-11-27 17:08:24 UTC] Score=100.00
[2025-11-27 17:1